In [90]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from torchvision import datasets
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

Data

The structure we expect is a 2x2x768 dim output

I used the other notebook, so i can have my functions, i'll implement them here too, if needed

Def functions

In [91]:
#it wants the sentences as a list already split to words/punctuation
#in the first column
def text_tokenizing(text_dataframe):
    input_ids=[]
    attention_masks=[]
    targets=[]
    for_preparing=tokenizer("")
    start_symbol=for_preparing.input_ids[0]
    end_symbol=for_preparing.input_ids[-1]
    
    for i in range(text_dataframe["sentence"].size):
        target_idx_counter=text_dataframe["target_idx"][i]
        input_ids_sentence=[]
        attention_masks_sentence=[]
        input_ids_sentence.append(start_symbol)
        attention_masks_sentence.append(1)
        
        for j in range(len(text_dataframe["sentence"][i])):
            
            #a word is text_daraframe.iloc[i,0][j]
            tokenized_word=tokenizer(
                                     text_dataframe["sentence"][i][j],
                                     add_special_tokens=False
                                    )
            
            input_id=tokenized_word.input_ids
            attention_mask=tokenized_word.attention_mask
                
            if(text_dataframe["sentence"][i][j]==text_dataframe["word"][i]):
                target=len(input_id)
                text_dataframe["target_idx"][i]=len(input_ids_sentence)           
            
            input_ids_sentence.extend(input_id)
            attention_masks_sentence.extend(attention_mask)
        
        input_ids_sentence.append(end_symbol)
        attention_masks_sentence.append(1)
        input_ids.append(input_ids_sentence)
        attention_masks.append(attention_masks_sentence)
        targets.append(target)
        
    #padding    
    max_length_ids=longest_sentence_size(input_ids)
    for i in range(len(input_ids)):
        current_size=len(input_ids[i])
        additional_zeros=[]
        for j in range(max_length_ids-current_size):
            additional_zeros.append(0)
        
        input_ids[i]=input_ids[i][0:-1]+additional_zeros+[input_ids[i][-1]]
        attention_masks[i]=attention_masks[i][0:-1]+additional_zeros+[attention_masks[i][-1]]
            
    tokenized_text=pd.DataFrame({
        'sentence': text_dataframe["sentence"],
        'word': text_dataframe["word"],
        'target_idx': text_dataframe["target_idx"],
        'target_length': targets,
        'type': text_dataframe["type"],
        'input_id': input_ids,
        'attention_mask': attention_masks
    })
    return tokenized_text

In [92]:
def model_output(data_dataframe):
    tokenized_data=text_tokenizing(data_dataframe)

    outputs=[]

    #every sentence gets turned into tensor
    #so we can let it through the mode, then save the output
    for i in range(tokenized_data['input_id'].size):
    
        sentence=[tokenized_data["input_id"][i],
              tokenized_data["attention_mask"][i]]
    
        tensor_sentence=torch.LongTensor(sentence)
    
        with torch.no_grad():
            output=model(
                tensor_sentence,
                output_hidden_states=True
            )
        
        outputs.append(output)

    first_half=[]
    second_half=[]

    for j in range(tokenized_data["input_id"].size):
        first_half.append(len(outputs[0][2][0][j]))
        second_half.append(len(outputs[0][2][1][j]))
                       
    output_beginning = pd.DataFrame({
        'first_half': first_half,
        'second_half': second_half
    })
                       
    return output_beginning

In [93]:
#we add 2 more to it, because we will use it to measure the sentences for
#the tokenizer, which has a starting and an end character additionally

def longest_sentence_size(sentences):
    current_max=0
    for sentence in sentences:
        if(len(sentence)>current_max):
            current_max=len(sentence)
    return current_max+2

Take datas out of files to pandas DataFrames

TODO: change it to a function

In [94]:
train_data = pd.read_csv(
    "train.tsv",
    sep='\t',
    names=['sentence',
           'word',
           'target_idx',
           'type'
          ]
)
print(train_data)
train_sentences = train_data.sentence.str.split(" ").to_frame()

train_data["sentence"]=train_sentences
train_data

                                               sentence         word  \
0     A szobát , ahol a 66 éves Kobajasi Sinicsi tet...       szobát   
1     Valami tudat alatt mindig működött bennem , va...     megérzés   
2     A megyében a legnagyobb problémát az átutazó f...     megyében   
3     A tíz kis , zömében szabolcsi konzervgyár hite...    hiteleire   
4     A 72 tagú konferencia által , három püspök fen...       püspök   
...                                                 ...          ...   
1949  Ez a műfaj lényege : olyan szuggesztivitással ...     operákat   
1950  Egyre nagyobb ugyanis a túlélők hatóságokkal s...  miniszterek   
1951   Ki ez ?  — kérdezte a magyar kommunisták első...  kommunisták   
1952  Az állami televíziófelügyelet például átmeneti...    némaságra   
1953                                  Ugatják a telet .        telet   

      target_idx  type  
0              1  Sing  
1              8  Sing  
2              1  Sing  
3             33  Plur  
4         

,sentence,word,target_idx,type
0,"[A, szobát, ,, ahol, a, 66, éves, Kobajasi, Si...",szobát,1,Sing
1,"[Valami, tudat, alatt, mindig, működött, benne...",megérzés,8,Sing
2,"[A, megyében, a, legnagyobb, problémát, az, át...",megyében,1,Sing
3,"[A, tíz, kis, ,, zömében, szabolcsi, konzervgy...",hiteleire,33,Plur
4,"[A, 72, tagú, konferencia, által, ,, három, pü...",püspök,7,Sing
...,...,...,...,...
1949,"[Ez, a, műfaj, lényege, :, olyan, szuggesztivi...",operákat,13,Plur
1950,"[Egyre, nagyobb, ugyanis, a, túlélők, hatóságo...",miniszterek,19,Plur
1951,"[, Ki, ez, ?, , —, kérdezte, a, magyar, kommun...",kommunisták,9,Plur
1952,"[Az, állami, televíziófelügyelet, például, átm...",némaságra,5,Sing


In [95]:
dev_data = pd.read_csv(
    "dev.tsv",
    sep='\t',
    names=['sentence',
           'word',
           'target_idx',
           'type'
          ]
)
dev_sentences = dev_data.sentence.str.split(" ").to_frame()
dev_data["sentence"] = dev_sentences
dev_data

,sentence,word,target_idx,type
0,"[Komoly, fegyvertény, például, ,, hogy, a, ház...",fegyvertény,1,Sing
1,"[Világosi, Gábor, szerint, a, megfigyelési, üg...",tényeket,22,Plur
2,"[A, olaj, világpiaci, árának, drasztikus, emel...",alakulására,16,Sing
3,"[Emellett, cél, olyan, nagy, tömegközlekedési,...",cél,1,Sing
4,"[A, Pedagógus, Szakszervezet, titkára, úgy, vé...",önkormányzatok,19,Plur
...,...,...,...,...
194,"[Eszerint, szükséges, lenne, ezek, törvényi, s...",esetben,9,Sing
195,"[Szerb, vádak, szerint, a, nemzetközi, közössé...",helyzetet,13,Sing
196,"[Véleménye, szerint, a, tervezett, új, jogszab...",forrásokat,15,Plur
197,"[A, múlt, keddi, kormányülést, követően, Járai...",számairól,19,Plur


In [96]:
dev_data

,sentence,word,target_idx,type
0,"[Komoly, fegyvertény, például, ,, hogy, a, ház...",fegyvertény,1,Sing
1,"[Világosi, Gábor, szerint, a, megfigyelési, üg...",tényeket,22,Plur
2,"[A, olaj, világpiaci, árának, drasztikus, emel...",alakulására,16,Sing
3,"[Emellett, cél, olyan, nagy, tömegközlekedési,...",cél,1,Sing
4,"[A, Pedagógus, Szakszervezet, titkára, úgy, vé...",önkormányzatok,19,Plur
...,...,...,...,...
194,"[Eszerint, szükséges, lenne, ezek, törvényi, s...",esetben,9,Sing
195,"[Szerb, vádak, szerint, a, nemzetközi, közössé...",helyzetet,13,Sing
196,"[Véleménye, szerint, a, tervezett, új, jogszab...",forrásokat,15,Plur
197,"[A, múlt, keddi, kormányülést, követően, Járai...",számairól,19,Plur


In [97]:
dev_xs = model_output(dev_data)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


IndexError: index 2 is out of bounds for dimension 0 with size 2

In [116]:
len(tokenized_dev_data["attention_mask"][0])

118

In [128]:
test_length=torch.LongTensor([tokenized_dev_data["input_id"],
                tokenized_dev_data["attention_mask"]])
test_length[0][5]

tensor([  101,   151, 73870, 10570, 13358, 10116,   169, 56011, 14205, 39374,
        10123,   117, 11352,   169,   179, 34871, 83653, 11250, 16459, 11969,
        14510, 12516, 79058, 10174, 29877, 11215, 10570, 84530, 10112, 14891,
          169, 14382, 22555, 79478, 15371, 10145, 18257, 25799, 15320, 34271,
        24817, 11481, 14139, 10443,   175, 23569,   117,   169, 11250, 16459,
        14205, 19248, 26770, 17221, 16621, 15538, 13632, 10360, 72295, 22367,
          179, 10333, 33616, 10161, 24830, 10347, 83291, 34138, 10116,   169,
        27452, 84530, 39832, 68077, 34871, 33482, 72295, 76030, 62123, 16530,
        45136, 10174, 88259, 26240, 10965, 49124, 58856, 41439, 83291, 11044,
          119,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,   102])

In [135]:
tokenized_dev_data=text_tokenizing(dev_data)
with torch.no_grad():
    test = tokenized_dev_data.apply(lambda x: model(
            torch.LongTensor([tokenized_dev_data["input_id"],
                tokenized_dev_data["attention_mask"]]),
            output_hidden_states=True
        ), axis=0)
test

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


sentence          [Emellett, cél, olyan, nagy, tömegközlekedési,...
word                                                            cél
target_idx                                                        3
target_length                                                     2
type                                                           Sing
input_id          [101, 11289, 76506, 171, 24817, 22361, 17221, ...
attention_mask    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
dtype: object

In [129]:
#WRONG
#
#
#

print(dev_data)
tokenized_dev_data=text_tokenizing(dev_data)
print(tokenized_dev_data)
print(dev_data)

outputs=[]

#every sentence gets turned into tensor
#so we can let it through the mode, then save the output
for i in range(tokenized_data['input_id'].size):
    
    sentence=[tokenized_data["input_id"][i],
              tokenized_data["attention_mask"][i]]
    
    
    tensor_sentence=torch.LongTensor(sentence)
    print(tensor_sentence.size())
    
    with torch.no_grad():
        output=model(
            tensor_sentence,
            output_hidden_states=True
        )
        
    outputs.append(output)

outputs

                                              sentence            word  \
0    [Komoly, fegyvertény, például, ,, hogy, a, ház...     fegyvertény   
1    [Világosi, Gábor, szerint, a, megfigyelési, üg...        tényeket   
2    [A, olaj, világpiaci, árának, drasztikus, emel...     alakulására   
3    [Emellett, cél, olyan, nagy, tömegközlekedési,...             cél   
4    [A, Pedagógus, Szakszervezet, titkára, úgy, vé...  önkormányzatok   
..                                                 ...             ...   
194  [Eszerint, szükséges, lenne, ezek, törvényi, s...         esetben   
195  [Szerb, vádak, szerint, a, nemzetközi, közössé...       helyzetet   
196  [Véleménye, szerint, a, tervezett, új, jogszab...      forrásokat   
197  [A, múlt, keddi, kormányülést, követően, Járai...       számairól   
198  [A, tanárok, fizetésének, pedig, ismét, el, ke...    későbbiekben   

     target_idx  type  
0             4  Sing  
1            51  Plur  
2            36  Sing  
3             3

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


                                              sentence            word  \
0    [Komoly, fegyvertény, például, ,, hogy, a, ház...     fegyvertény   
1    [Világosi, Gábor, szerint, a, megfigyelési, üg...        tényeket   
2    [A, olaj, világpiaci, árának, drasztikus, emel...     alakulására   
3    [Emellett, cél, olyan, nagy, tömegközlekedési,...             cél   
4    [A, Pedagógus, Szakszervezet, titkára, úgy, vé...  önkormányzatok   
..                                                 ...             ...   
194  [Eszerint, szükséges, lenne, ezek, törvényi, s...         esetben   
195  [Szerb, vádak, szerint, a, nemzetközi, közössé...       helyzetet   
196  [Véleménye, szerint, a, tervezett, új, jogszab...      forrásokat   
197  [A, múlt, keddi, kormányülést, követően, Járai...       számairól   
198  [A, tanárok, fizetésének, pedig, ismét, el, ke...    későbbiekben   

     target_idx  target_length  type  \
0             4              4  Sing   
1            51              3 

[(tensor([[[-0.0502,  0.0355, -0.1117,  ...,  0.0203,  0.1488, -0.0266],
           [-0.1621,  0.0679, -0.0299,  ...,  0.3956, -0.1598, -0.0519],
           [-0.0623, -0.9058,  0.3047,  ..., -0.0486,  0.7122,  0.1871],
           ...,
           [ 0.6373, -0.8077,  1.4583,  ...,  0.5731,  1.3505,  0.0200],
           [ 0.5378, -0.6330,  1.4126,  ...,  0.5571,  1.1021, -0.0407],
           [ 0.0819, -0.3707,  0.2414,  ..., -0.1837,  0.1091, -0.2038]],
  
          [[ 0.2923, -0.4480,  1.0444,  ...,  0.4133, -0.3787, -0.2371],
           [ 0.0911, -0.5461,  1.0113,  ...,  0.4790, -0.0890, -0.2799],
           [ 0.0959, -0.5524,  0.9027,  ...,  0.4963, -0.1360, -0.3392],
           ...,
           [ 0.3189, -0.7956,  0.7657,  ...,  0.9040, -0.0186, -0.1196],
           [ 0.3366, -0.7804,  0.7653,  ...,  0.9013, -0.0563, -0.1288],
           [ 0.3389, -0.7126,  0.7444,  ...,  0.8540, -0.0616, -0.1658]]]),
  tensor([[ 0.1606, -0.0443,  0.1853,  ..., -0.1711,  0.1000,  0.0906],
          [-0

In [76]:
len(tokenized_data["input_id"][0])

118

In [89]:
len(outputs)

199

In [294]:
first_half = pd.DataFrame({})
for i in range(len(outputs)):
    first_half.insert(i, i, outputs[i][2][0][0][(tokenized_data["target_idx"][i]
        + tokenized_data["target_length"][i]
        - 1)], True)

In [295]:
type(first_half[0][0])

numpy.float32

In [296]:
train_x = torch.tensor(first_half.values)

In [297]:
type(train_x)

torch.Tensor

In [262]:
#WRONG
#
#

first_half_outputs = outputs[0:204][2][0]
first_halves = outputs[0:204][2][0][0][(tokenized_data["target_idx"][0:204]
        + tokenized_data["target_length"][0:204]
        - 1)]
first_halves.size()
second_halves = outputs[0:204][2][0][0][(tokenized_data["target_idx"][0:204]
        + tokenized_data["target_length"][0:204]
        - 1)]
type(second_halves[0][1])

torch.Tensor

In [264]:
len(outputs)

204

In [267]:
test_idxs

0       9
1      25
2      43
3      32
4       4
       ..
199    10
200    40
201    42
202    75
203    15
Length: 204, dtype: int64

I decided to use only the first half.


It's because using both could make the sizes confusing,
or the training less understandable

In [298]:
type(data_to_prepare["type"].unique())
labels = tokenized_data["type"].unique()
type(labels)

numpy.ndarray

In [299]:
text_train_y = tokenized_data["type"]

labels = data_to_prepare["type"].unique()

print(labels)
print(type(text_train_y))

text_train_y

['Sub' 'Sup' 'Nom' 'Ine' 'Acc' 'Ins']
<class 'pandas.core.series.Series'>


0      Sub
1      Sup
2      Nom
3      Ine
4      Nom
      ... 
199    Nom
200    Acc
201    Ins
202    Ine
203    Ine
Name: type, Length: 204, dtype: object

In [300]:
#transforming train_y to a series of number by the labels
train_y=[]

for i in range(text_train_y.size):
    train_y.append(np.where(labels == text_train_y[i])[0][0])
    
train_y

[0,
 1,
 2,
 3,
 2,
 4,
 1,
 3,
 4,
 4,
 3,
 4,
 0,
 4,
 0,
 2,
 1,
 3,
 5,
 0,
 3,
 3,
 0,
 2,
 1,
 0,
 5,
 2,
 1,
 1,
 3,
 5,
 5,
 3,
 3,
 5,
 5,
 3,
 4,
 2,
 2,
 5,
 0,
 2,
 5,
 3,
 3,
 4,
 4,
 5,
 2,
 1,
 4,
 4,
 5,
 3,
 1,
 4,
 4,
 4,
 1,
 1,
 2,
 0,
 4,
 4,
 0,
 3,
 4,
 0,
 3,
 2,
 2,
 4,
 1,
 1,
 0,
 0,
 4,
 5,
 0,
 2,
 2,
 5,
 3,
 0,
 5,
 3,
 0,
 3,
 0,
 4,
 5,
 5,
 1,
 3,
 3,
 0,
 3,
 1,
 1,
 1,
 2,
 4,
 0,
 0,
 2,
 3,
 2,
 2,
 3,
 5,
 5,
 2,
 5,
 2,
 2,
 2,
 5,
 3,
 1,
 5,
 5,
 0,
 2,
 1,
 5,
 0,
 4,
 5,
 1,
 0,
 5,
 3,
 4,
 0,
 0,
 5,
 4,
 1,
 5,
 0,
 2,
 1,
 1,
 1,
 5,
 4,
 4,
 1,
 4,
 3,
 2,
 4,
 1,
 4,
 2,
 1,
 0,
 5,
 4,
 2,
 1,
 0,
 5,
 0,
 0,
 2,
 5,
 1,
 1,
 4,
 1,
 0,
 5,
 3,
 1,
 2,
 4,
 2,
 5,
 3,
 1,
 3,
 5,
 3,
 4,
 2,
 2,
 2,
 3,
 0,
 4,
 3,
 0,
 0,
 0,
 1,
 1,
 2,
 4,
 5,
 3,
 3]

In [301]:
train_y=torch.tensor(train_y)

In [302]:
print(type(train_x[1]), type(train_y))


<class 'torch.Tensor'> <class 'torch.Tensor'>


In [303]:
train_y

tensor([0, 1, 2, 3, 2, 4, 1, 3, 4, 4, 3, 4, 0, 4, 0, 2, 1, 3, 5, 0, 3, 3, 0, 2,
        1, 0, 5, 2, 1, 1, 3, 5, 5, 3, 3, 5, 5, 3, 4, 2, 2, 5, 0, 2, 5, 3, 3, 4,
        4, 5, 2, 1, 4, 4, 5, 3, 1, 4, 4, 4, 1, 1, 2, 0, 4, 4, 0, 3, 4, 0, 3, 2,
        2, 4, 1, 1, 0, 0, 4, 5, 0, 2, 2, 5, 3, 0, 5, 3, 0, 3, 0, 4, 5, 5, 1, 3,
        3, 0, 3, 1, 1, 1, 2, 4, 0, 0, 2, 3, 2, 2, 3, 5, 5, 2, 5, 2, 2, 2, 5, 3,
        1, 5, 5, 0, 2, 1, 5, 0, 4, 5, 1, 0, 5, 3, 4, 0, 0, 5, 4, 1, 5, 0, 2, 1,
        1, 1, 5, 4, 4, 1, 4, 3, 2, 4, 1, 4, 2, 1, 0, 5, 4, 2, 1, 0, 5, 0, 0, 2,
        5, 1, 1, 4, 1, 0, 5, 3, 1, 2, 4, 2, 5, 3, 1, 3, 5, 3, 4, 2, 2, 2, 3, 0,
        4, 3, 0, 0, 0, 1, 1, 2, 4, 5, 3, 3])

In [310]:
train_x = torch.transpose(train_x, 0,1)

In [312]:
print("train size:", train_x.size(), train_y.size())

train size: torch.Size([204, 768]) torch.Size([204])


Random indexes

In [313]:
all_idx = np.arange(len(train_y))
np.random.shuffle(all_idx)
train_idx = all_idx

Batching Class

In [314]:
class BatchIterator:
    def __init__(self, x, y, batch_size):
        self.x = x
        self.y = y
        self.batch_size = batch_size
        
    def iterate_once(self):
        for start in range(0, len(self.x), self.batch_size):
            end = start + self.batch_size
            yield self.x[start:end], self.y[start:end]

Test the iterator:

In [315]:
train_iter = BatchIterator(train_x, train_y, 60)

for batch in train_iter.iterate_once():
    print(batch[0].size(), batch[1].size())

torch.Size([60, 768]) torch.Size([60])
torch.Size([60, 768]) torch.Size([60])
torch.Size([60, 768]) torch.Size([60])
torch.Size([24, 768]) torch.Size([24])


The classifier

First try

In [316]:
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim):
        super().__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, X):
        h = self.input_layer(X)
        h = self.relu(h)
        out = self.output_layer(h)
        return out

Model Instance

In [317]:
model = Classifier(
    input_dim = train_x.size(1),
    output_dim = labels.size,
    hidden_dim = 50
)
model

Classifier(
  (input_layer): Linear(in_features=768, out_features=50, bias=True)
  (relu): ReLU()
  (output_layer): Linear(in_features=50, out_features=6, bias=True)
)

In [318]:
for n, p in model.named_parameters():
    print(n, p.size())

input_layer.weight torch.Size([50, 768])
input_layer.bias torch.Size([50])
output_layer.weight torch.Size([6, 50])
output_layer.bias torch.Size([6])


Loss function and optimizer

In [319]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

Testing out the performance

At this point it should be around random so around 10%

In [320]:
test_pred = model(train_x).max(axis=1)[1]
test_acc = torch.eq(test_pred, train_y).sum().float() / len(train_x)
test_acc

tensor(0.1961)

Training Loop

In [321]:
batch_size = 50
train_iter = BatchIterator(train_x, train_y, batch_size)